## RAG System Using Llama2 

In [1]:
!pip install pypdf

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [3]:
pip install openai


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [5]:
## Embedding
!pip install install sentence_transformers

In [6]:
!pip install llama_index

from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [17]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

In [15]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
pip install llama_index.llms.huggingface

documents=SimpleDirectoryReader("TAGR.pdf").load_data()
documents

In [21]:
import fitz 
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [22]:
pdf_path = 'TAGR.pdf'
pdf_text = extract_text_from_pdf(pdf_path)


system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name=" "))

In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
service_context

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("how can we get success according to think and grow rich?")

In [ ]:
print(response)

In [ ]:
response=query_engine.query("who is author of think and grow rich?")

In [ ]:
print(response)